In [40]:
!pip install tensor-dash

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/Agro/plant/train.csv')

In [11]:
!pip install tensorflow
!pip install streamlit
!pip install matplotlib
!pip install pandas
!pip install scikit-learn


In [12]:
# Import necessary libraries
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to files
train_csv_path = '/content/drive/MyDrive/Agro/plant/train.csv'
test_csv_path = '/content/drive/MyDrive/Agro/plant/test.csv'
image_folder = '/content/drive/MyDrive/Agro/plant/images/'  # Update this if the images are in a specific folder

# Load the train.csv and test.csv
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Preview the dataset
print("Train Data:")
print(train_df.head())

# Splitting the training data into train and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")


Train Data:
  image_id  healthy  multiple_diseases  rust  scab
0  Train_0        0                  0     0     1
1  Train_1        0                  1     0     0
2  Train_2        1                  0     0     0
3  Train_3        0                  0     1     0
4  Train_4        1                  0     0     0
Training samples: 1456, Validation samples: 365


In [13]:
import os

# List all image filenames in the folder
image_filenames = set(os.listdir(image_folder))

# Check for mismatched filenames
mismatched_filenames = [img for img in train_df['image_id'] if img not in image_filenames]

if mismatched_filenames:
    print(f"Number of mismatched filenames: {len(mismatched_filenames)}")
    print("Examples of mismatched filenames:", mismatched_filenames[:10])
else:
    print("All filenames match!")


Number of mismatched filenames: 1821
Examples of mismatched filenames: ['Train_0', 'Train_1', 'Train_2', 'Train_3', 'Train_4', 'Train_5', 'Train_6', 'Train_7', 'Train_8', 'Train_9']


In [14]:
import os

# List the first few filenames to check their extensions
image_folder = '/content/drive/MyDrive/Agro/plant/images/'  # Update this path if necessary
example_files = os.listdir(image_folder)[:10]
print("Example filenames in the folder:", example_files)


Example filenames in the folder: ['Test_648.jpg', 'Train_1308.jpg', 'Test_593.jpg', 'Test_1760.jpg', 'Test_1803.jpg', 'Test_1148.jpg', 'Test_115.jpg', 'Test_1771.jpg', 'Test_1766.jpg', 'Train_1260.jpg']


In [15]:
# Add '.jpg' extension to all filenames in train and val data
train_data['image_id'] = train_data['image_id'].apply(lambda x: f"{x}.jpg")
val_data['image_id'] = val_data['image_id'].apply(lambda x: f"{x}.jpg")

# Check updated filenames
print("Updated train data image_id examples:", train_data['image_id'].head())


Updated train data image_id examples: 1455    Train_1455.jpg
1552    Train_1552.jpg
300      Train_300.jpg
1708    Train_1708.jpg
611      Train_611.jpg
Name: image_id, dtype: object


In [16]:
import os

# Define the image folder path
image_folder = '/content/drive/MyDrive/Agro/plant/images/'  # Adjust if needed

# Get a set of all filenames in the folder
image_filenames = set(os.listdir(image_folder))

# Check for missing files
missing_files = [img for img in train_data['image_id'] if img not in image_filenames]
if missing_files:
    print(f"Number of missing files: {len(missing_files)}")
    print("Examples of missing files:", missing_files[:10])
else:
    print("All filenames match!")


All filenames match!


In [45]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

# Ensure labels are strings
train_data['labels'] = train_data['labels'].astype(str)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/drive/MyDrive/Agro/plant/images/',
    x_col='image_id',
    y_col='labels',  # Ensure labels are now strings
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Change to binary classification
    subset='training',
    shuffle=True,
    workers=4,
    use_multiprocessing=True
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='/content/drive/MyDrive/Agro/plant/images/',
    x_col='image_id',
    y_col='labels',  # Ensure labels are now strings
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Change to binary classification
    subset='validation',
    shuffle=True,
    workers=4,
    use_multiprocessing=True
)


Found 1165 validated image filenames belonging to 2 classes.
Found 291 validated image filenames belonging to 2 classes.


In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='softmax')  # 4 classes: healthy, multiple_diseases, rust, scab
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()  # Print the model architecture


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_25 (Dropout)                 │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 186624)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 128)                 │      23,888,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,907,521 (91.20 MB)

 Trainable params: 23,907,521 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# Check the first batch from the train generator
train_batch = next(train_generator)
print(f"Train batch images shape: {train_batch[0].shape}")
print(f"Train batch labels shape: {train_batch[1].shape}")

# Check the first batch from the validation generator
val_batch = next(val_generator)
print(f"Validation batch images shape: {val_batch[0].shape}")
print(f"Validation batch labels shape: {val_batch[1].shape}")


Train batch images shape: (32, 224, 224, 3)
Train batch labels shape: (32,)
Validation batch images shape: (32, 224, 224, 3)
Validation batch labels shape: (32,)


In [60]:
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 164s 4s/step - accuracy: 0.2922 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 0.0000e+00
Epoch 2/10
 1/36 ━━━━━━━━━━━━━━━━━━━━ 1:39 3s/step - accuracy: 0.3125 - loss: 0.0000e+00

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 124ms/step - accuracy: 0.3125 - loss: 0.0000e+00 - val_accuracy: 0.6667 - val_loss: 0.0000e+00
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 183s 4s/step - accuracy: 0.2851 - loss: 0.0000e+00 - val_accuracy: 0.2535 - val_loss: 0.0000e+00
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2188 - loss: 0.0000e+00 - val_accuracy: 0.3333 - val_loss: 0.0000e+00
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 196s 4s/step - accuracy: 0.2873 - loss: 0.0000e+00 - val_accuracy: 0.2569 - val_loss: 0.0000e+00
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2812 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 207s 4s/step - accuracy: 0.2871 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 0.0000e+00
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3438 - loss: 0.0000e+00 - val_accuracy: 0.6667 - val_loss: 0.0000e+00
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 191s 4s/step - accuracy: 0

In [61]:
model.save('/content/drive/MyDrive/Agro/plant/model.h5')


In [67]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.models import load_model
import numpy as np

# Define image folder and image size
image_folder = "/content/drive/MyDrive/Agro/plant/images"  # Adjust this path if needed
IMG_SIZE = (224, 224)  # Adjust size based on your model's expected input

# Load the test data CSV file
test_df = pd.read_csv('/content/drive/MyDrive/Agro/plant/test.csv')

# Ensure image_id in CSV has extensions if missing
test_df['image_id'] = test_df['image_id'] + '.jpg'  # Adjust this if your images are .png or another format

# Check if image filenames match and print out
print(test_df['image_id'].head())  # Ensure the first few image IDs match your actual filenames

# Preprocess test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=image_folder,
    x_col='image_id',
    target_size=IMG_SIZE,
    batch_size=1,
    shuffle=False,
    class_mode=None  # No labels, as this is for predictions
)

# Load the trained model (make sure the path is correct)
model = load_model('/content/drive/MyDrive/Agro/plant/model.h5')

# Predict on test data
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)

# Check the shape of predictions to verify that it matches the expected output
print(predictions.shape)  # Should output something like (num_samples, 1)

# If predictions are a single column with class indices (0-3), directly convert them to one-hot encoding
if predictions.shape[1] == 1:
    # Flatten predictions and convert them to class indices
    predicted_classes = predictions.flatten().astype(int)
    predictions_reshaped = np.zeros((predictions.shape[0], 4))
    for i in range(predictions.shape[0]):
        predictions_reshaped[i, predicted_classes[i]] = 1
else:
    # If predictions are already one-hot encoded, use them as-is
    predictions_reshaped = predictions

# Prepare submission (Ensure predictions have the same number of columns as your classes)
submission_df = test_df.copy()

# Since your model output has 4 columns (healthy, multiple_diseases, rust, scab), ensure the predictions are assigned correctly
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = predictions_reshaped

# Save the final submission
submission_df.to_csv('/content/drive/MyDrive/Agro/plant/sample_submission.csv', index=False)
print("Submission file saved!")


0    Test_0.jpg
1    Test_1.jpg
2    Test_2.jpg
3    Test_3.jpg
4    Test_4.jpg
Name: image_id, dtype: object
Found 1821 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1821/1821 ━━━━━━━━━━━━━━━━━━━━ 103s 56ms/step
(1821, 1)
Submission file saved!


In [69]:
!pip install streamlit

In [73]:
!pip install pyngrok

In [81]:
!pip install tensorflow Pillow numpy
